In [1]:
import sys
import numpy as np
import pandas as pd
from utils import *

In [2]:
df_preds, models_by_state, data_all = load_preds()

In [3]:
df_preds.model_id.unique()

array(['A', 'B', 'C', 'D', 'E', 'F', 'G', 'ln_base', 'ln_crps', 'ln_log',
       'lin_crps', 'lin_log'], dtype=object)

In [13]:
df_preds_all.model_id.unique()

array([21, 22, 25, 27, 28, 30, 34])

In [14]:
df_preds_problem = df_preds_all.loc[df_preds_all.pred <= 0.1]
    
excluded_models = list(set(zip(df_preds_problem.model_id, df_preds_problem.state)))

excluded_models

[(21, 'RR'),
 (21, 'MG'),
 (25, 'MG'),
 (21, 'AL'),
 (25, 'CE'),
 (21, 'ES'),
 (21, 'RO')]

In [2]:
df_preds_all.head()

,date,pred,lower,upper,state,model_id
0,2022-10-09,110.690113,64.291013,219.826948,AC,21
1,2022-10-16,144.952846,77.564037,279.722708,AC,21
2,2022-10-23,163.613937,92.915072,276.591399,AC,21
3,2022-10-30,170.240443,87.197631,378.407811,AC,21
4,2022-11-06,215.117888,111.835667,445.024983,AC,21


### Applying the CRPS

In [3]:
%%time 
df_for_val_base = pd.DataFrame()

df_24 = pd.DataFrame()

df_25 = pd.DataFrame()

df_w = pd.DataFrame()

metric = 'crps'
dist = 'log_normal'
for state in states_BR: 
    
    data_23, data_24, preds_23, preds_24, preds_25, models = format_data(state, models_by_state, data_all, df_preds_all)
    
    ens_24 = Ensemble(df= preds_24,
            order_models = models,  dist = dist)

    weights_24 = ens_24.compute_weights(df_obs=data_24, metric = metric)

    df_24_out = ens_24.apply_ensemble(weights = weights_24['weights'])

    df_24 = pd.concat([df_24, df_24_out], ignore_index = True)

CPU times: user 4min 45s, sys: 1.28 s, total: 4min 47s
Wall time: 5min 1s


In [6]:
state

'PR'

In [7]:
df_24_out.head()

,pred,lower,upper,date
0,1004.019798,569.720083,1769.387783,2023-10-08
1,1116.551167,650.027768,1917.897311,2023-10-15
2,1151.715265,676.906657,1959.573064,2023-10-22
3,1215.908779,721.248241,2049.827055,2023-10-29
4,1358.572768,826.657046,2232.751751,2023-11-05


In [11]:
df2 = pd.read_csv('../predictions/ensemble_out_sample_2024.csv.gz')
df2 = df2.loc[df2.state == state]
df2.head()

,date,pred,lower,upper,state
910,2023-10-08,1004.019798,569.720083,1769.387783,PR
911,2023-10-15,1116.551167,650.027768,1917.897311,PR
912,2023-10-22,1151.715265,676.906657,1959.573064,PR
913,2023-10-29,1215.908779,721.248241,2049.827055,PR
914,2023-11-05,1358.572768,826.657046,2232.751751,PR
